In [2]:
import numpy as np
pi = np.pi
pi2i = pi*2j
from matplotlib import pyplot as plt
from time import time

In [3]:
# some aux functions
def cplx2tuple(z):
    return np.real(z), np.imag(z)

# intro
I solved the interior 2d stokes problem on the circle in this jupyter notebook, which serves as a basis for further solution of this problem

In [4]:
# number of points
n = 1000

# parameterization of the curve
a = np.linspace(0,2*np.pi,n+1)[:-1]
h = 2*np.pi / n

# complex coordinates of points 
t = np.exp(a*1j)

# parametrization derivatives
t_dt = t*1j

# boundary conditions
v = np.zeros(n)
u = 1-(t.imag)**2
h2 = u
h1 = -v
H = np.ones(n)

# curvature
curvature = np.ones(n)

the blue dots is equal-space pts on the boundary $\Gamma=S^1$, the red line is vector field of velocity, the green line is the tangential derivatives. the yellow line is the normal derivatives


The velocity is the normalized presoille flow with integration = 1

The kernels and BI eq can be greatly simplified in this specific case, compare to the original case in L. Greengard's paper. In this case, it is 

$$
K_1'(t_i,t_j) = \frac{-h}{\pi} \Im(\frac{d_j}{dt})\\
K_2'(t_i,t_j) = \frac{-h}{2\pi i} (\frac{-d_j}{\bar{dt}} + \frac{\bar{d_j}dt}{\bar{dt}^2}) 
$$ 
where $dt = t_i-t_j$, $d_j$ is the derivative of the parametrization. 

And in the limiting case of $i=j$, the kernels should be 

$$
K_1'(t_i,t_i) = \frac{h}{2\pi}\kappa_i |d_i| \\
K_2'(t_i,t_i) = \frac{-h}{2\pi} \kappa_i d_i^2/ |d_i|
$$

In [5]:
def K1_func(i,j):
    if i == j:
        k = curvature[i]
        d = np.abs(t_dt[i])
        return h*k*d/(2*np.pi)
    dt = t[i] - t[j]
    d = t_dt[j]
    return -h*np.imag(d/dt)/np.pi

def K2_func(i,j):
    if i==j:
        k = curvature[i]
        d = np.abs(t_dt[i])
        return h*k*d/(2*np.pi)
    d = t_dt[j]
    dt = t[i] - t[j]
    return -h/(2j*pi) * (-d/np.conjugate(dt) + np.conjugate(d)*dt/np.conjugate(dt**2))

In [6]:
# start timing for building solver
t1 = time()

In [7]:
dt = t[:,np.newaxis] - t[np.newaxis,:]
with np.errstate(divide='ignore', invalid='ignore'):
    temp = t_dt[np.newaxis,:] / dt
    temp = temp.imag
    K1 = -h/pi * temp
    K2 = -h/(2j*pi) * (-t_dt[np.newaxis,:]/np.conjugate(dt) + np.conjugate(t_dt)[np.newaxis,:]*dt/np.conjugate(dt**2))
for i in range(n):
    K1[i,i] = K1_func(i,i)
    K2[i,i] = K2_func(i,i)

this calculation can be vectorized for speeding up, but I guess it needs to handle the case of i == j more carefully. 

My integral equation does not contain any singular term, this could lead to inaccuracy and convergence, I guess. 

Should I transfer this into a real equation for the solution?? Or is it I can just live with the complex equation? How to deal with the complex conjugates though... 

The integral equation is of the following form

$$
(I+K_1+K_2\mathfrak C) \omega = h
$$ where $\mathfrak C$ is a symbol for conjugation. 

This can be discreticized in the the points of our choice and then seperate the real and imaginary matrix equation so that 

$$
\begin{pmatrix}
I+\Re(K_1+K_2) & \Im(-K_1+K_2) \\
\Im(K_1 + K_2) & I+\Re(K_1-K_2)
\end{pmatrix}
\begin{pmatrix}
\Re \omega\\
\Im \omega
\end{pmatrix} = 
\begin{pmatrix}
\Re h\\
\Im h
\end{pmatrix}
$$

In [8]:
A = np.zeros((2*n,2*n))
A[:n,:n] = np.identity(n) + (K1+K2).real
A[:n,n:] = (-K1+K2).imag
A[n:,:n] = (K1+K2).imag
A[n:,n:] = np.identity(n) + (K1-K2).real

In [9]:
rhs = np.concatenate((h1,h2))

In [10]:
omega = np.linalg.solve(A,rhs)
omega = omega[:n] + 1j*omega[n:]

In [11]:
# start timing for evaluations
t2 = time()

In [12]:
def evaluation(z):
    
    t_z = t-z
    t_z_sq = t_z**2
    c = h/(pi2i)
    omega_times_dt = omega*t_dt
    
    phi = c*np.sum(omega_times_dt/t_z)
    d_phi = c*np.sum(omega_times_dt/(t_z_sq))
    
    psi = c*(
        2*np.sum((np.conjugate(t_dt)*omega).real/t_z)
        -np.sum(np.conjugate(t)*omega_times_dt/t_z_sq))
    
    dW = phi + z*np.conjugate(d_phi) + np.conjugate(psi)
    
    return dW

In [23]:
def evaluation_(z):
    
    omega = np.ones(1)
    
    t_z = t-z
    t_z_sq = t_z**2
    c = h/(pi2i)
    omega_times_dt = omega*t_dt
    
    phi = c*np.sum(omega_times_dt/t_z)
    d_phi = c*np.sum(omega_times_dt/(t_z_sq))
    
    psi = c*(
        2*np.sum((np.conjugate(t_dt)*omega).real/t_z)
        -np.sum(np.conjugate(t)*omega_times_dt/t_z_sq))
    
    dW = phi + z*np.conjugate(d_phi) + np.conjugate(psi)
    
    return dW

In [13]:
grid_density = 100
radius = 0.95 
# evaluations close to radius 1 would result in catastrophic errors, therefore I will evaluate for a radius a bit smaller than 1
circle_grid = np.array([(x,y) for x in np.linspace(-1,1,grid_density) for y in np.linspace(-1,1,grid_density) if x**2 + y**2 <= radius**2])

In [14]:
circle_grid_X, circle_grid_Y = circle_grid.T
circle_grid_z = circle_grid_X + 1j*circle_grid_Y

In [15]:
circle_grid_values = np.array([evaluation(z) for z in circle_grid_z])

In [16]:
true_values = 1j*np.array([1-y**2 for x,y in circle_grid])

In [17]:
# computation ended
t3 = time()

# performance


In [18]:
print('condition number of the matrix')
np.linalg.cond(A)

condition number of the matrix


2003.006010022094

In [22]:
np.mean(circle_grid_values)

(-1.2212144033224305e-17+0.7740936381285033j)

In [19]:
error = (np.abs(circle_grid_values - true_values))
relative_error = error/np.abs(true_values)

In [39]:
print('relative error for velocity: mean and max')
np.max(error), np.mean(error)

relative error for velocity: mean and max


(0.00033730376064598376, 0.00012302276458277223)

In [40]:
print('relative error for velocity: mean and max')
np.max(relative_error), np.mean(relative_error)

relative error for velocity: mean and max


(0.002656493163354959, 0.00020666114319948202)

In [42]:
print('total time consumed')
t3-t2

total time consumed


0.2983982563018799

In [43]:
print('time for building the solver over ',n,' boundary points')
t2-t1

time for building the solver over  1000  boundary points


0.23461461067199707

In [44]:
print('time for evaluating the solver over ',len(circle_grid_z),' interior points')
t3-t2

time for evaluating the solver over  6956  interior points


0.2983982563018799